<a href="https://colab.research.google.com/github/anuragsingh17ai/Natural-Language-Processing/blob/main/01_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=ISNdQcPhsts&t=1303s

# Transformer Implementation

## 1. Input Embeddings

Input embedding converts Words into 512 dimension Vector embeddings

* It will take input
* then It will map these inputs to Input IDs(position in the vocabulary)
* then It will then converted into vector of size 512

In [1]:
import torch
from torch import nn
import math


In [2]:
# d_model = size of vector in our case it is 512
# vocab_size = size of vocab dic
class InputEmbeddings(nn.Module):
  def __init__(self,d_model:int, vocab_size:int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self,x):
    return self.embedding(x) * math.sqrt(self.d_model)






## 2. Positional encoding

positional Embedding are of same size as of embedding. It is basically used to assing position of
every word so model can remember it's sequence

<img src="https://github.com/anuragsingh17ai/Natural-Language-Processing/blob/main/images/01_transformer_images/positional%20Embeddings.png?raw=true" alt = "postional embeddings" width= 900/>

In [3]:
# seq_len = len of sentence
# sin is used for even position
# cos is used for odd position

class PositionalEncoding(nn.Module):
  def __init__(self,d_model:int, seq_len: int , dropout: float)-> None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    # Create a matrix of shape (seq_len, d_model)
    pe = torch.zeros(seq_len, d_model)
    # Create a vector of shape (seq_len, 1)
    position = torch.arange(0,seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0,d_model,2).float()*(-math.log(1000.0)/d_model))

    #Apply the sin to even position
    pe[:,0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    pe = pe.unsqueeze(0) # (1, seq_len, d_model)

    self.register_buffer('pe', pe) # register_buffer helps to save info in model when you don't want to train model on that info

  def forward(self,x):
    x = x + (self.pe[:,:x.shape[1],:]).requires_grad(False)
    return self.dropout(x)


In [ ]:
class LayerNormalization(nn.Module):
  def __init__(self,eps:float, )